# Solucion de ecuaciones lineales $Ax=\lambda x$ o $Ax=b$ por medio de DMRG

## Planteamiento del problema

*  Considere un operador $H$ de orden $N$, con dimensiones locales $d$
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/H_diagram.png" width="400">
<br>
*  El algoritmo DMRG busca el eigenvector dominante de $H$ en la forma MPS, donde $E_0$ ($\le E_1\le E_2$...) es el menor valor propio de $H$
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/H_eigenvector.png" width="400">
<br>
*  Para que el algoritmo sea eficiente $H$ debe estar en la forma MPO
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/H_MPO_form.png" width="400">

## Pasos del algoritmo DMRG

### Paso 0: Preparacion

*  Llevar al MPS inicial a una forma ortogonal (Aqui escogemos que sea ortogonal a derecha)
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/right_ortho_MPS.png" width="400">
<br>
*  Por la propiedad de ortogonalizacion a derecha, podemos interpretar los tensores MPS 3,4,... colectivamente como un cambio de base desde la base de indices locales $i_3$,$i_4$,... a la de indice de enlace $\alpha_2$
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/MPS_change_of_basis.png" width="400">
<br>
*  Esta interpretacion motiva a la transformacion de $H$ a la base $i_1$,$i_2$,$\alpha_2$ 
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/projected_H.png" width="400">
<br>
*  Tomando $H$ como MPO, podemos calcular la transformacion eficientemente, definiendo los tensores $R_j$ por el camino
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/H_edge.png" width="400">
<br>
*  Por eficiencia, Es crucial que los tensores de borde $R_j$ se creen contrayendo cada tensor MPS o MPO uno a la vez en un cierto orden
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/H_edge_ordering.png" width="400">
<br>
*  Luego, contraemos los dos primeros tensores MPS
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/bond1.png" width="400">
<br>

**Observacion:**
*  El tensor de enlace $B_{12}^{i_1 i_2 \alpha}$ representa enteramente al eigenvetor de $H$ solo que escrito en la base $i_1$,$i_2$,$\alpha_2$ 
*  Si la base $\alpha_2$ abarca un subespacio de $i_3$, $i_4$, ..., $i_N$ espacio suficiente para representar el vector propio de $H$ que buscamos, entonces optimizar $B_{12}$ para que sea el vector propio dominante del $H$ transformado es suficiente para resolver el problema original.
*  En la práctica, sin embargo, se puede mejorar la base $\alpha_2$, y para hacerlo, el algoritmo optimiza el MPS en cada enlace secuencialmente.

### Paso 1a: Optimizacion del primer tensor de enlace

*  Para optimizar $B_{12}$ los algoritmos mas eficientes son Davidson or Lanczos
*  El paso clave de cada uno de estos algoritmos es la multiplicación de $B_{12}$ por la matriz (transformada) $H$. Usando la forma proyectada de $H$ en términos de los tensores $R_j$ definidos anteriormente
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/B12_H_mult.png" width="400">
<br>
*  El algoritmo iterativo toma el tensor $H.B_{12}$ resultante y lo procesa aún más, dependiendo del algoritmo particular utilizado. Los pasos de multiplicación de la misma forma que el diagrama anterior se repiten hasta que se cumpla algún criterio de convergencia. Estos pasos de multiplicación generalmente dominan el costo de computo del DMRG. El resultado es un tensor de enlace mejorado $B'_{12}$ que se aproxima más al vector propio dominante de $H$ que se busca.

**Observacion:**
*  Para un algoritmo DMRG eficiente, uno no debería hacer converger completamente el algoritmo iterativo de solución en este paso, o los siguientes pasos similares de optimización de tensores de enlaces. Hasta que todo el algoritmo converja globalmente, no se puede suponer que el subespacio abarcado por el resto de los tensores MPS sea el ideal para representar el vector propio verdadero.

### Paso 1b: Restauración adaptativa del formato MPS del tensor de enlace

*  Para que el algoritmo sea eficiente, es necesario restaurar la forma del tensor de enlace mejorado $B'_{12}$
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/B12_svd.png" width="400">
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/SV_mult.png" width="400">
<br>
*  El tensor del indice $i_1$ queda en la forma canonica a izquierda
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/left_ortho.png" width="400">
<br>
*  La descomposicion produce un MPS optimo
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/MPS_dist.png" width="400">
<br>

### Paso 2: Optimizacion y adaptacion del segundo tensor de enlace

<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/B23.png" width="400">
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/L1.png" width="400">
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/mult_H_B23.png" width="400">
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/B23_SVD.png" width="400">
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/M3.png" width="400">
<br>
<img src="https://tensornetwork.org/mps/algorithms/dmrg/L2.png" width="400">
<br>

In [ ]:
# -*- coding: utf-8 -*-
"""
The system we consider is the quantum XX-model, which is closely related to the Bose-Hubbard model.


mainDMRG_MPO.py
---------------------------------------------------------------------
Script file for initializing the Hamiltonian of a 1D spin chain as an MPO \
before passing to the DMRG routine.

    by Glen Evenbly (c) for www.tensors.net, (v1.1) - last modified 19/1/2019
"""

#### Preamble
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

from ncon import ncon
from doDMRG_MPO import doDMRG_MPO 

##### Example 1: XX model #############
#######################################

##### Set bond dimensions and simulation options
chi = 16;
Nsites = 50;

OPTS_numsweeps = 4 # number of DMRG sweeps
OPTS_dispon = 2 # level of output display
OPTS_updateon = True # level of output display
OPTS_maxit = 2 # iterations of Lanczos method
OPTS_krydim = 4 # dimension of Krylov subspace

#### Define Hamiltonian MPO (quantum XX model)
chid = 2
sP = np.sqrt(2)*np.array([[0, 0],[1, 0]])
sM = np.sqrt(2)*np.array([[0, 1],[0, 0]])
sX = np.array([[0, 1], [1, 0]]) # pauli matrices
sY = np.array([[0, -1j], [1j, 0]]) #puali matrices
sZ = np.array([[1, 0], [0,-1]]) #pauli matrices
sI = np.array([[1, 0], [0, 1]]) #identity
M = np.zeros([4,4,chid,chid]);
M[0,0,:,:] = sI; M[3,3,:,:] = sI
M[0,1,:,:] = sM; M[1,3,:,:] = sP
M[0,2,:,:] = sP; M[2,3,:,:] = sM
ML = np.array([1,0,0,0]).reshape(4,1,1) #left MPO boundary
MR = np.array([0,0,0,1]).reshape(4,1,1) #right MPO boundary

#### Initialize MPS tensors
A = [0 for x in range(Nsites)] # tensores de la red MPS
A[0] = np.random.rand(1,chid,min(chi,chid))
for k in range(1,Nsites):
    A[k] = np.random.rand(A[k-1].shape[2],chid,min(min(chi,A[k-1].shape[2]*chid),chid**(Nsites-k-1)))

#### Do DMRG sweeps (2-site approach)
En1, Ap, sWeight, B = doDMRG_MPO(A,ML,M,MR,chi, numsweeps = OPTS_numsweeps, dispon = OPTS_dispon, 
                                updateon = OPTS_updateon, maxit = OPTS_maxit, krydim = OPTS_krydim)

#### Increase bond dim and reconverge
chi = 32;
En2, App, sWeight, B = doDMRG_MPO(A,ML,M,MR,chi, numsweeps = OPTS_numsweeps, dispon = OPTS_dispon, 
                                updateon = OPTS_updateon, maxit = OPTS_maxit, krydim = OPTS_krydim)

#### Compare with exact results (computed from free fermions)
H = np.diag(np.ones(Nsites-1),k=1) + np.diag(np.ones(Nsites-1),k=-1)
D = LA.eigvalsh(H)
EnExact = 2*sum(D[D < 0])

##### Plot results
plt.figure(1)
plt.yscale('log')
plt.plot(range(len(En1)), En1 - EnExact, 'b', label="chi = 16")
plt.plot(range(len(En2)), En2 - EnExact, 'r', label="chi = 32")
plt.legend()
plt.title('DMRG for XX model')
plt.xlabel('Update Step')
plt.ylabel('Ground Energy Error')
plt.show()

